In [1]:
import numpy as np, pandas as pd

In [2]:
import numbers, unittest

In [3]:
class BaseValidator:
    def __init__(self, minimum=None, maximum=None):
        self._min = minimum
        self._max = maximum
    
    def __set_name__(self, owner, name):
        self.owner = owner
        self.name = name
    
    def __get__(self, obj, type):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)
    
    def __set__(self, obj, value):
        self.validate(value)
        obj.__dict__[self.name] = value
    
    def validate(self, value):
        pass

In [4]:
class IntegerField(BaseValidator):
    
    def validate(self, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'{self.name} must be an integer')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.name} must be at least {self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.name} must be no larger than {self._max}')

In [5]:
class CharField(BaseValidator):
    def __init__(self, minimum=None, maximum=None):
        minimum = max(minimum or 0, 0)
        super().__init__(minimum, maximum)
    
    def validate(self, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.name} must be a string')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.name} must be at least {self._min} long')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.name} must be no longer than {self._max}')

In [6]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
        
    def test_set_age_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_invalid(self):
        """Tests that invalid values raise ValueErrors"""
        min_ = -10
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_values = list(range(min_ - 5, min_))
        bad_values += list(range(max_ + 1, max_ + 5))
        bad_values += [10.5, 1 + 0j, 'abc', (1, 2)]
        
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age, IntegerField)
        
    def test_set_age_min_only(self):
        """Tests that we can specify a min value only"""
        min_ = 0
        max_ = None
        obj = self.create_test_class(min_, max_)
        values = range(min_, min_ + 100, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_max_only(self):
        """Tests that we can specify a max value only"""
        min_ = None
        max_ = 10
        obj = self.create_test_class(min_, max_)
        values = range(max_ - 100, max_, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_no_limits(self):
        """Tests that we can use IntegerField without any limits at all"""
        min_ = None
        max_ = None
        obj = self.create_test_class(min_, max_)
        values = range(-100, 100, 10)
        for i, value in enumerate(values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

class TestCharField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'name': CharField(min_, max_)})
        return obj()
        
    def test_set_name_ok(self):
        """Tests that valid values can be assigned/retrieved"""
        min_ = 1
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(min_, max_)
        
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
            
    def test_set_name_invalid(self):
        """Tests that invalid values raise ValueErrors"""
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_lengths = list(range(min_ - 5, min_))
        bad_lengths += list(range(max_ + 1, max_ + 5))
        for i, length in enumerate(bad_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.name = value
                    
    def test_class_get(self):
        """Tests that class attribute retrieval returns the descriptor instance"""
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.name, CharField)
        
    def test_set_name_min_only(self):
        """Tests that we can specify a min length only"""
        min_ = 0
        max_ = None
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(min_, min_ + 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
    
    def test_set_name_min_negative_or_none(self):
        """Tests that setting a negative or None length results in a zero length"""
        obj = self.create_test_class(-10, 100)
        self.assertEqual(type(obj).name._min, 0)
        self.assertEqual(type(obj).name._max, 100)
        
        obj = self.create_test_class(None, None)
        self.assertEqual(type(obj).name._min, 0)
        self.assertIsNone(type(obj).name._max)
        
    def test_set_name_max_only(self):
        """Tests that we can specify a max length only"""
        min_ = None
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(max_ - 100, max_, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
                
    def test_set_name_no_limits(self):
        """Tests that we can use CharField without any limits at all"""
        min_ = None
        max_ = None
        obj = self.create_test_class(min_, max_)
        valid_lengths = range(0, 100, 10)
        for i, length in enumerate(valid_lengths):
            value = 'a' * length
            with self.subTest(test_number=i):
                obj.name = value
                self.assertEqual(value, obj.name)
                
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [7]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField)
Tests that invalid values raise ValueErrors ... ok
test_set_age_max_only (__main__.TestIntegerField)
Tests that we can specify a max value only ... ok
test_set_age_min_only (__main__.TestIntegerField)
Tests that we can specify a min value only ... ok
test_set_age_no_limits (__main__.TestIntegerField)
Tests that we can use IntegerField without any limits at all ... ok
test_set_age_ok (__main__.TestIntegerField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.075s

OK


In [8]:
run_tests(TestCharField)

test_class_get (__main__.TestCharField)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_name_invalid (__main__.TestCharField)
Tests that invalid values raise ValueErrors ... ok
test_set_name_max_only (__main__.TestCharField)
Tests that we can specify a max length only ... ok
test_set_name_min_negative_or_none (__main__.TestCharField)
Tests that setting a negative or None length results in a zero length ... ok
test_set_name_min_only (__main__.TestCharField)
Tests that we can specify a min length only ... ok
test_set_name_no_limits (__main__.TestCharField)
Tests that we can use CharField without any limits at all ... ok
test_set_name_ok (__main__.TestCharField)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.077s

OK
